# Video Decode Unit (VDU) Demo Example: STREAM_IN-> DECODE -> FAKESINK 

# Introduction

Video Decode Unit (VDU) in Versal SOC is capable of decoding AVC/HEVC compressed video streams in real time. 

This notebook example acts as Client pipeline in streaming use case. It needs to be run along with Server. It receives encoded data over network, decode using VDU and render it on DP/HDMI Monitor.


# Implementation Details

<img src="pictures/block-diagram-streamin-decode.png" align="center" alt="Drawing" style="width: 600px; height: 200px"/>

This example requires two boards, board-1 is used for transcode and stream-out (as a server) and **board 2** is used for streaming-in and decode purpose (as a client) or VLC player on the host machine can be used as client instead of board-2 (More details regarding Test Setup for board-1 can be found in transcode → stream-out Example).

__Note:__ This notebook needs to be run along with "vcu-demo-transcode-to-streamout.ipynb" or "vcu-demo-camera-encode-streamout.ipynb". The configuration settings below are for Client-side pipeline. 

### Board Setup
**Board 2 is used for streaming-in and decode purpose (as a client)**
 1. Connect 4k DP/HDMI display to board.
 2. Connect serial cable to monitor logs on serial console.
 3. If Board is connected to private network, then export proxy settings in /home/root/.bashrc file on board as below,
    - create/open a bashrc file using "vi ~/.bashrc" 
        - Insert below line to bashrc file
            - export http_proxy="< private network proxy address >"
            - export https_proxy="< private network proxy address >"
        - Save and close bashrc file.
 4. Connect two boards in the same network so that they can access each other using IP address.
 5. Check server IP on server board.
    - root@zcu106-zynqmp:~#ifconfig
 6. Check client IP.
 7. Check connectivity for board-1 & board-2.
    - root@zcu106-zynqmp:~#ping <board-2's IP>
 8. Run stream-in → Decode on board-2
 
Create test.sdp file on host with below content (Add separate line in test.sdp for each item below) and play test.sdp on host machine.
 1. v=0 c=IN IP4 <Client machine IP address>
 2. m=video 50000 RTP/AVP 96
 3. a=rtpmap:96 H264/90000
 4. a=framerate=30
    
Trouble-shoot for VLC player setup:
 1. IP4 is client-IP address
 2. H264/H265 is used based on received codec type on the client
 3. Turn-off firewall in host machine if packets are not received to VLC.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Run the Demo

In [2]:
from ipywidgets import interact
import ipywidgets as widgets
from common import common_vdu_demo_streamin_decode_display
import os
from ipywidgets import HBox, VBox, Text, Layout

### Video

In [3]:
import ipywidgets as widgets

class VideoSection(widgets.HBox):
    def __init__(self, codec_options=['avc', 'hevc'], sink_options={'kmssink': ['DP', 'HDMI'], 'fakevideosink': ['none']}, decoder_options=['allegrodecIP0', 'allegrodecIP1', 'allegrodecIP2', 'allegrodecIP3']):
        
        # Initialize parent VBox
        super().__init__()
        
        
        # Initialize codec options
        self.codec_options = codec_options
        self.codec_type = widgets.RadioButtons(
            options=self.codec_options,
            description='Video Codec:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        self.codec_type.observe(self._update_codec_type, names='value')
        self.selected_codec = self.codec_type.value
        
        
        # Initialize video sink options
        self.sink_options = sink_options
        self.sink_type = widgets.RadioButtons(
            options=sorted(self.sink_options.keys(), key=lambda k: len(self.sink_options[k]), reverse=True),
            value='fakevideosink',
            description='Video Sink:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        init = self.sink_type.value
        self.display_type = widgets.RadioButtons(
            #options=self.sink_options[self.sink_type.value],
            options=self.sink_options[init], # Set default value to 'none'
            description='Display:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        # Initialize stream options
        self.decoder_options = decoder_options
        self.decoder_instance = widgets.RadioButtons(
            options=self.decoder_options,
            description='Dec Instance:',
            style={'description_width': 'auto', 'button_width': '10px'},
        )
        #init1 = self.stream_type.value
        #self.stream_num = widgets.RadioButtons(
            #options=self.stream_options[self.stream_type.value],
         #   options=self.stream_options[init1],
          #  description='Instance:',
           # style={'description_width': 'auto', 'button_width': '10px'}
        #)
        
        # Define widget layout
        self.layout = widgets.Layout(border='')
        
        # Add widgets to VBox
        self.children = [self.codec_type, self.sink_type, self.display_type, self.decoder_instance]
        
        # Attach update functions
        self.sink_type.observe(self._update_display_options, names='value')
        #self.stream_type.observe(self._update_stream_options, names='value')
        
    def _update_display_options(self, change):
        self.display_type.options = self.sink_options[self.sink_type.value]
        
    #def _update_stream_options(self, change):
        #self.stream_num.options = self.stream_options[self.stream_type.value]
        
    #def get_codec_type(self):
       # return self.codec_type.value
        
    def _update_codec_type(self, change):
        self.selected_codec = self.codec_type.value
        #self.selected_codec = change.new
        
    def get_codec_type(self):
        return self.codec_type.value
        
        
# Create first video section
video1 = VideoSection()
#video_sections = [video1]

# Display first video section
display(video1)





VideoSection(children=(RadioButtons(description='Video Codec:', options=('avc', 'hevc'), style=DescriptionStyl…

### Advanced options:

In [4]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Text, Layout
class AdvanceOption(widgets.HBox):
   def __init__(self):
      super().__init__()
      
      self.kernel_recv_buffer_size = widgets.Text(value='',
         placeholder='(optional) 16000000',
         description='Kernel Recv Buf Size:',
         style={'description_width': 'initial'},
         #layout=Layout(width='33%', height='30px'),
         disabled=False)
      self.port_number=widgets.Text(value='',
         placeholder='(optional) 50000, 42000',
         description=r'Port No:',
         #style={'description_width': 'initial'},
         #
         disabled=False)
      
      self.children = [self.kernel_recv_buffer_size, self.port_number]
      self.layout = widgets.Layout(border='')
   
   def get_kernel_recv_buffer_size(self):
      return self.kernel_recv_buffer_size.value
   
   def get_port_number(self):
      return self.port_number.value
   
#kernel_recv_buffer_size
advance_option = AdvanceOption()

display(advance_option)

AdvanceOption(children=(Text(value='', description='Kernel Recv Buf Size:', placeholder='(optional) 16000000',…

In [5]:
# Add a new video section
button = widgets.Button(description='Add more decoder', button_style='success')
#def add_video_section(button):
  #  video2 = VideoSection()
   # display(video2)

# Define a counter for the number of video sections
count = 1

# Initialize empty lists of input and video sections
advance_options = [advance_option]
#proxy_sections = [proxy_section]
video_sections = [video1]



# Display button section
#display(widgets.VBox([video_container, button_widget]))
# Define the add_video_section function with the counter
def add_video_section(button):
    global count
    name = f'video{count+1}'
    
    # Create new instances of InputSection and VideoSection
    A_Option = AdvanceOption()
    #proxy = ProxySection()
    video = VideoSection()
    
    #video._dom_classes = [name]  # set class name to object name
    #display(video)
    count += 1
    
    # Add the new instances to the existing lists
    advance_options.append(A_Option)
    #proxy_sections.append(proxy)
    video_sections.append(video)
    
    
    #display(proxy)
    display(video)
    display(A_Option)
    
    # Remove the previous button
    button.layout.display = 'none'
    
    # Add a new button below the new video instance
    new_button = widgets.Button(description='Add more decoder', button_style='success')
    new_button.on_click(add_video_section)
    if count <= 3:
        display(new_button)


button.on_click(add_video_section)
display(button)


Button(button_style='success', description='Add more decoder', style=ButtonStyle())

In [6]:
entropy_buffers=widgets.Dropdown(
    options=['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    value='5',
    description='Entropy Buffers Nos:',
    style={'description_width': 'initial'},
    disabled=False,)
show_fps=widgets.Checkbox(
    value=False,
    description='show-fps',
    #style={'description_width': 'initial'},
    disabled=False)
HBox([entropy_buffers, show_fps])

In [7]:
from IPython.display import clear_output
from IPython.display import Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

def clear_op(event):
    clear_output(wait=True)
    return

button1 = widgets.Button(
    description='Clear Output',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '300px'}
)
button2 = widgets.Button(
    description='',
    style= {'button_color':'white'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '38px'},
    disabled=True
)
button1.on_click(run_all)
button1.on_click(clear_op)

In [9]:
import subprocess 
import time

def start_demo(event):
    audio_codec = 'none'
    audio_sink = 'none'
    #clear_output(wait=True)
    print_process = 0
    arg = [];
    arg1 = [];
    arg2 = [];
    arg3 = [];
    arg = common_vdu_demo_streamin_decode_display.cmd_line_args_generator(advance_option.port_number.value, video1.codec_type.value, audio_codec, video1.display_type.value, advance_option.kernel_recv_buffer_size.value, video1.sink_type.value, entropy_buffers.value, show_fps.value, audio_sink, video1.decoder_instance.value);
    cmd1 = "sh vdu-demo-streamin-decode-display.sh {} 2>&1".format(arg)
    process1 = subprocess.Popen(cmd1, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    if count >= 2:
        arg1 = common_vdu_demo_streamin_decode_display.cmd_line_args_generator(advance_options[1].port_number.value, video_sections[1].codec_type.value, audio_codec, video_sections[1].display_type.value, advance_options[1].kernel_recv_buffer_size.value, video_sections[1].sink_type.value, entropy_buffers.value, show_fps.value, audio_sink, video_sections[1].decoder_instance.value);
        cmd2 = "sh vdu-demo-streamin-decode-display.sh {} 2>&1".format(arg1)
        process2 = subprocess.Popen(cmd2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    if count >= 3:
        arg2 = common_vdu_demo_streamin_decode_display.cmd_line_args_generator(advance_options[2].port_number.value, video_sections[2].codec_type.value, audio_codec, video_sections[2].display_type.value, advance_options[2].kernel_recv_buffer_size.value, video_sections[2].sink_type.value, entropy_buffers.value, show_fps.value, audio_sink, video_sections[2].decoder_instance.value);
        cmd3 = "sh vdu-demo-streamin-decode-display.sh {} 2>&1".format(arg2)
        process3 = subprocess.Popen(cmd3, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    if count >= 4:
        arg3 = common_vdu_demo_streamin_decode_display.cmd_line_args_generator(advance_options[3].port_number.value, video_sections[3].codec_type.value, audio_codec, video_sections[3].display_type.value, advance_options[3].kernel_recv_buffer_size.value, video_sections[3].sink_type.value, entropy_buffers.value, show_fps.value, audio_sink, video_sections[3].decoder_instance.value);
        cmd4 = "sh vdu-demo-streamin-decode-display.sh {} 2>&1".format(arg3)
        process4 = subprocess.Popen(cmd4, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    for line in process1.stdout:
        if print_process == 1:
            print("----------------->Process 1 output:<---------------")
        print(line.strip())
        if count >= 2:
            print_process = 1
            print("----------------->Process 2 Output:<---------------")
            for line in process2.stdout:
                print(line.strip())
                break
        if count >= 3:
            print("----------------->Process 3 Output:<---------------")
            for line in process3.stdout:
                print(line.strip())
                break
        if count >= 4:
            print("----------------->Process 4 Output:<---------------")
            for line in process4.stdout:
                print(line.strip())
                break
    return

button = widgets.Button(
    description='click to start vdu-stream_in-decode-display demo',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '350px'}
)
button.on_click(start_demo)
HBox([button, button2, button1])

sh common_vdu_demo_streamin_decode_display.sh -p 5004 -c hevc -o fakevideosink -f -z allegrodecIP0
No input buffer size specified hence using 60000000 as default size for kernel recieved buffer
gst-launch-1.0 udpsrc port=5004 buffer-size=60000000 caps="application/x-rtp, media=video, clock-rate=90000, payload=96, encoding-name=H265" ! rtpjitterbuffer latency=1000 ! queue ! rtph265depay ! h265parse ! omxh265dec device=/dev/allegroDecodeIP0 low-latency=1 ! queue max-size-bytes=0 ! fpsdisplaysink name=fpssink text-overlay=false video-sink="fakevideosink" -v
Setting pipeline to PAUSED ...
Pipeline is live and does not need PREROLL ...
/GstPipeline:pipeline0/GstFPSDisplaySink:fpssink/GstFakeVideoSink:fakevideosink0/GstFakeSink:sink: sync = true
Pipeline is PREROLLED ...
Setting pipeline to PLAYING ...
/GstPipeline:pipeline0/GstUDPSrc:udpsrc0.GstPad:src: caps = application/x-rtp, media=(string)video, clock-rate=(int)90000, payload=(int)96, encoding-name=(string)H265
/GstPipeline:pipeline0/Gs

# References
[1] https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842546/Xilinx+Video+Codec+Unit

[2] https://www.xilinx.com/support.html#documentation (Refer to PG252)